In [2]:
from core.db import get_engine
from core.libs import pd, text, np

In [5]:
# =====================================================================
# Extraer solo la tabla de relaciones (df_master) sin exportar nada
# =====================================================================

from core.db import get_engine
from core.libs import pd, text, Path

# Importa las funciones necesarias del archivo del usuario
from MasterDatabaseManagement.Exports.masterdatabase_architecture_join_master import extract_dependencies, SCHEMA

# Conexión
engine = get_engine()

print("=" * 70)
print("🔗 GENERANDO DATAFRAME DE RELACIONES (df_master)")
print("=" * 70)

# Extraer TODAS las relaciones (physical + logical + trigger + manual)
df_master = extract_dependencies(engine, filter_type=None)

# Enriquecer columnas igual que en main(), pero sin guardar
df_master["schema_name"] = SCHEMA
df_master["last_audit"] = pd.Timestamp.now().normalize()
df_master["dependency_depth"] = df_master["relation_type"].map({
    "physical": 0,
    "logical": 1,
    "trigger": 2,
    "manual": 3
}).fillna(99).astype(int)

# Ordenar por tipo y tabla
df_master = df_master.sort_values(["dependency_depth", "parent_table"])

print("\n📄 DataFrame RESULTANTE:")
print(df_master.to_string(index=False))

print("\nTotal relaciones:", len(df_master))
print("\nColumnas:", list(df_master.columns))


💻 Conectado a la base de datos helloworldtree
🔗 GENERANDO DATAFRAME DE RELACIONES (df_master)

📄 DataFrame RESULTANTE:
                      parent_table                                child_table                          link_name relation_type    schema_name last_audit  dependency_depth
masterdatabase.contract_allocation   masterdatabase.contract_tree_information                          ca_cti_fk      physical masterdatabase 2025-11-18                 0
    masterdatabase.contract_header masterdatabase.farmer_personal_information contract_header_farmer_number_fkey      physical masterdatabase 2025-11-18                 0
            fpi_contracts_expanded                farmer_personal_information                    view_dependency       logical masterdatabase 2025-11-18                 1
                  survival_current                  inventory_metrics_current                    view_dependency       logical masterdatabase 2025-11-18                 1
        vw_atlas_etp_year_